# Instalar la libreria

In [ ]:
pip install openai

# Importa la clase OpenAI

In [1]:
from openai import OpenAI

# Crea una instancia del cliente

In [2]:
openai = OpenAI(api_key="Ingresa tu API Key de OpenAI") 

# **Qué hace:**

1. Llama al endpoint de **chat completions**:

   * `model='gpt-3.5-turbo'`: define el modelo.
   * `messages=[...]`: define la conversación:

     * `system`: le dice al modelo cómo comportarse (“asistente que da información…”).
     * `user`: la pregunta del usuario.
   * `max_tokens=50`: limita la longitud de la respuesta (aprox. cantidad de “palabras/pedazos” que puede generar; técnicamente son tokens).

2. Guarda la respuesta en `response`.

3. Imprime el texto final con:

   * `response.choices[0].message.content`
   * `choices[0]` es la primera (y usualmente la única) respuesta generada.


In [ ]:
response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content" :"Eres un asistente que da informacion a dudas"},
        {"role": "user", "content" :"¿Quién descubrió América?"}
    ],
    max_tokens = 50,
)

print(response.choices[0].message.content)

# Ejemplo practico

In [ ]:
from openai import OpenAI
import os

# Recomendado: guardar la API Key como variable de entorno
# En Windows PowerShell:  $env:OPENAI_API_KEY="tu_api_key"
# En Linux/Mac:           export OPENAI_API_KEY="tu_api_key"

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Eres un asistente experto en deportes. Responde claro y con datos puntuales."},
        {"role": "user", "content": "¿Quién ganó el mundial de fútbol de 2022 y en qué país se jugó?"}
    ],
    temperature=0.7,   # más variación que 0.2–0.3
    max_tokens=120,
    n=2                # pide 2 alternativas de respuesta
)

for i, choice in enumerate(response.choices, start=1):
    print(f"Respuesta {i}:\n{choice.message.content}\n")


# Ejemplo 2

In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Eres un asistente experto en deportes. Responde claro y breve."},
        {"role": "user", "content": "¿En qué país se jugó el mundial de fútbol de 2022?"}
    ],
    temperature=0.3,
    max_tokens=120,
    stream=True
)

# Imprime el texto a medida que llega (tipo “escribiendo…”)
for chunk in stream:
    # En streaming, el texto viene por partes (deltas)
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end="", flush=True)

print()  # salto de línea al final



## ¿Qué hace este código?

1) `stream=True`

Cuando agregas `stream=True`, el modelo **no devuelve toda la respuesta completa de una sola vez**.
En lugar de eso, devuelve un **iterador** (un “flujo”) de fragmentos (chunks) con partes del texto.

2) `for chunk in stream:`

* Recorre cada fragmento que llega desde la API.
* Cada `chunk` trae una porción pequeña de la respuesta.

3) `chunk.choices[0].delta.content`

* En modo streaming, la salida no está en `message.content` como antes.
* Llega en un “delta” (pedacito incremental). Por eso se usa:

  * `chunk.choices[0].delta.content`
* Algunos chunks no traen texto (pueden traer señales de control), por eso se valida con `if`.

4) `print(..., end="", flush=True)`

* `end=""` evita que imprima salto de línea en cada fragmento.
* `flush=True` fuerza a que se muestre inmediatamente en la salida del notebook.

---

## Diferencia con el caso normal (`stream=False`)

**Sin streaming (normal)**

* Esperas a que termine toda la generación.
* Accedes al texto así:

```python
response = client.chat.completions.create(...)
print(response.choices[0].message.content)
```

**Con streaming (`stream=True`)**

* Vas recibiendo la respuesta por pedazos.
* Accedes al texto así:

```python
for chunk in stream:
    print(chunk.choices[0].delta.content, end="")
```

---

**¿Cuándo conviene usar `stream=True`?**

* Cuando quieres una UX tipo “escribiendo…”.
* Cuando las respuestas pueden ser largas y quieres ver progreso.
* Cuando quieres empezar a procesar texto antes de que termine (por ejemplo, mostrarlo en interfaz).



# Ejemplo 3

**Conceptos clave (qué controla cada parámetro)**

**`max_tokens`**

* Define la **longitud máxima** de la respuesta generada.
* Si lo pones muy bajo, la respuesta puede quedar **cortada** o demasiado breve.
* Si lo aumentas, permites respuestas más extensas (aunque no “obligas” al modelo a llenar todo; solo amplías el límite).

**Ejemplo:**

* `max_tokens=20` → una o dos frases cortas.
* `max_tokens=150` → explicación más completa con contexto.

---

**`temperature`**

* Controla la **creatividad/variabilidad** del modelo.
* **Baja temperatura (0.0–0.3):** más predecible, consistente, “conservadora”.
* **Alta temperatura (0.8–1.2):** más variada, con más libertad de redacción y detalles (y también mayor riesgo de “irse por las ramas” o agregar cosas no solicitadas).

**Regla práctica:**

* Si quieres respuestas “tipo dato” o muy estables → usa temperatura baja.
* Si quieres alternativas, redacción distinta, ideas → sube la temperatura.

---

**`top_p`**

* Controla la diversidad mediante **nucleus sampling**.
* Funciona como un “filtro probabilístico”: el modelo considera solo el conjunto de tokens cuya probabilidad acumulada llega a `top_p`.
* **`top_p` bajo (ej. 0.2–0.5):** limita la gama de tokens considerados → menos variabilidad.
* **`top_p` alto (cerca de 1):** deja pasar más opciones → mayor diversidad.

**Importante:** en la práctica se suele ajustar **temperature o top_p**, no ambos a la vez, para entender claramente el efecto de cada uno.

---

**`n`**

* Indica cuántas **variantes de respuesta** quieres que devuelva el modelo.
* Si `n=1`, recibes una sola respuesta.
* Si `n=3`, recibes tres alternativas en `response.choices`.

Esto es muy útil cuando:

* quieres comparar estilos,
* buscas diferentes formas de explicar lo mismo,
* o deseas escoger la “mejor” respuesta entre varias.

---

**¿Qué deberías observar en los resultados?**

1) Efecto de `max_tokens`

- **Caso 1 (max_tokens=25):**

  * Probablemente una respuesta de una sola línea.
  * Ejemplo esperado: “Cristóbal Colón en 1492.”
  * Si el modelo intenta añadir contexto, puede cortarse.

-  **Caso 2 (max_tokens=150):**

  * Puede explicar con más detalle:

    * quién fue Colón,
    * año,
    * patrocinio (Corona de Castilla),
    * matices como “llegada a América” vs “descubrimiento” (enfoque histórico).

**Idea importante:** `max_tokens` no cambia “la verdad”, cambia el *espacio disponible* para explicarla.

---

2) Efecto de `temperature`

- **Caso 3 (temperature=0.1):**

  * Respuesta más directa, parecida entre ejecuciones repetidas.
  * Poca variación en palabras.

- **Caso 4 (temperature=1.0):**

  * Puede cambiar la redacción entre ejecuciones:

    * “Cristóbal Colón llegó a América en 1492…”
    * “El viaje liderado por Colón…”
    * puede incluir más contexto o matices, incluso si no lo pediste.
  * Mayor diversidad en cómo explica.

**Resumen:**

* Baja temperatura: coherente, conservadora.
* Alta temperatura: variada, menos predecible.

---

3) Efecto de `top_p`

* **Caso 5 (top_p=0.3):**

  * El modelo se limita a tokens “muy probables”.
  * Normalmente verás menos creatividad en vocabulario y menos caminos alternativos.
  * La respuesta suele ser más “recta”, con menos adornos.

**¿Qué cambia al alterar top_p?**

* **top_p bajo**: reduce la gama de salidas posibles, por lo tanto baja variabilidad.
* **top_p alto (1.0)**: permite más diversidad (siempre en combinación con el muestreo).

---

 4) Efecto de `n`

* **Caso 6 (n=3):**

  * El modelo devuelve 3 alternativas.
  * Puede que las tres digan el mismo dato principal, pero con:

    * diferente estilo (más breve vs más explicativo),
    * distinto enfoque (histórico, educativo, neutral),
    * o variaciones de redacción.

Esto es útil para:

* elegir la respuesta que mejor encaje con tu audiencia,
* comparar consistencia,
* o validar si el modelo mantiene el dato central en varias opciones.

---

**Recomendaciones prácticas**

- Si quieres respuestas **consistentes y tipo “dato”**:

  * `temperature=0.0–0.3`
  * `top_p=1.0` (o no tocarlo)
  * `n=1`
  * `max_tokens` ajustado al tamaño esperado (ej. 60–120)

- Si quieres **varias alternativas** para escoger:

  * `n=3` (o más, con moderación)
  * `temperature=0.7` (para que no sean casi iguales)
  * `max_tokens` suficiente para que haya diferencias reales

- Si notas demasiada variación o “adornos”:

  * baja `temperature` **o** baja `top_p`
  * (mejor cambiar uno solo para ver claramente el efecto)




In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def run_case(title, max_tokens=80, temperature=0.2, top_p=1.0, n=1):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Eres un asistente educativo. Responde con claridad y precisión."},
            {"role": "user", "content": "¿Quién descubrió América?"}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        n=n
    )

    print("=" * 80)
    print(f"{title}")
    print(f"max_tokens={max_tokens} | temperature={temperature} | top_p={top_p} | n={n}")
    print("-" * 80)

    for i, choice in enumerate(response.choices, start=1):
        print(f"[Respuesta {i}] {choice.message.content}\n")


# Caso 1: Respuesta corta (max_tokens bajo)
run_case(
    title="Caso 1: max_tokens bajo (respuesta corta)",
    max_tokens=25,
    temperature=0.2,
    top_p=1.0,
    n=1
)

# Caso 2: Respuesta más larga (max_tokens alto)
run_case(
    title="Caso 2: max_tokens alto (más explicación)",
    max_tokens=150,
    temperature=0.2,
    top_p=1.0,
    n=1
)

# Caso 3: Temperatura baja (más estable)
run_case(
    title="Caso 3: temperatura baja (más predecible)",
    max_tokens=120,
    temperature=0.1,
    top_p=1.0,
    n=1
)

# Caso 4: Temperatura alta (más variabilidad)
run_case(
    title="Caso 4: temperatura alta (más variación en estilo y detalle)",
    max_tokens=120,
    temperature=1.0,
    top_p=1.0,
    n=1
)

# Caso 5: top_p bajo (diversidad restringida) con temperatura moderada
run_case(
    title="Caso 5: top_p bajo (reduce la diversidad del texto)",
    max_tokens=120,
    temperature=0.7,
    top_p=0.3,
    n=1
)

# Caso 6: Variantes múltiples (n > 1)
run_case(
    title="Caso 6: n=3 (tres alternativas para elegir)",
    max_tokens=120,
    temperature=0.7,
    top_p=1.0,
    n=3
)
